# Implied volatility

## Abstract 


- We will introduce implied volatility and its computations

## Problem

We have seen that BS call price is 5.94 with the following parameters:

In [0]:
vol_ratio = .2; spot_price = 100.; drift_ratio = .0475; strike = 110.; maturity = 1.



In the market, usually volatility (vol = 0.2 in the above) is not available, but market price (call = 5.94) for the option is available. One wants to find a volatility such that associated BSM formula is equal to (fit) the market price. This is called implied volatility. 

Next, we are goint to find implied volatility given the following parameters:

spot_price = 100.; drift_ratio = .0475; strike = 110.; maturity = 1.; market_call_price = 5.94

## Anal

the implied volatility of an option contract is that value of the volatility of the underlying instrument which, when input in an option pricing model (such as Black–Scholes) will return a theoretical value equal to the current market price of the option. 

Consider a derivative or a call option particularly, we know parameter sets of 
- spot price.; drift_ratio (interest rate); strike.; maturity

For a BSM theoretical price, the only missing parameter is the volatility $\sigma$. Let's denote a function $f$ be
$$f: \sigma \mapsto BSM(\sigma, others).$$
Suppose we denote by $p$ for the market price of the call option, then the implied volatility is
$\hat \sigma$ satisfying
$$f(\hat \sigma) = p$$
or 
$$\hat \sigma = f^{-1}(p),$$
if the inverse of $f$ is well defined at $p$.

The next lemma tells us, we can search for the minimizer of the error function
$$\sigma \mapsto |f(\sigma) - p|,$$
which is available by "scipy.optimize.fmin". 

__Lemma__

Supose $f$ is a function satisfying
- $f: [0, \infty) \mapsto (f_{min}, f_{max})$
- $f$ is continuous
- $f$ is strictly increasing

then, for any $p\in  (f_{min}, f_{max})$, 
- there exists unique $\hat \sigma$, s.t. $f(\hat \sigma) = p$;
- Moreover, 
$$\hat \sigma = \arg\min_{\sigma\in (0,\infty)} | f(\sigma) - p|.$$

## Code

For this purpose, we first need bsm pricing engines developed before.

In [0]:
import numpy as np
import scipy.stats as ss

'''=========
option class init
=========='''
class VanillaOption:
    def __init__(
        self,
        otype = 1, # 1: 'call'
                  # -1: 'put'
        strike = 110.,
        maturity = 1.,
        market_price = 10.):
      self.otype = otype
      self.strike = strike
      self.maturity = maturity
      self.market_price = market_price #this will be used for calibration
      
        
    def payoff(self, s): #s: excercise price
      otype = self.otype
      k = self.strike
      maturity = self.maturity
      return max([0, (s - k)*otype])
'''============
Gbm class
============='''

class Gbm:
    def __init__(self,
                 init_state = 100.,
                 drift_ratio = .0475,
                 vol_ratio = .2
                ):
        self.init_state = init_state
        self.drift_ratio = drift_ratio
        self.vol_ratio = vol_ratio

'''========
Black-Scholes-Merton formula. 
=========='''

def bsm_price(self, vanilla_option):
    s0 = self.init_state
    sigma = self.vol_ratio
    r = self.drift_ratio
    
    otype = vanilla_option.otype
    k = vanilla_option.strike
    maturity = vanilla_option.maturity
    
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma ** 2) 
          * maturity) / (sigma * np.sqrt(maturity))
    d2 = d1 - sigma * np.sqrt(maturity)
    
    return (otype * s0 * ss.norm.cdf(otype * d1) #line break needs parenthesis
            - otype * np.exp(-r * maturity) * k * ss.norm.cdf(otype * d2))

Gbm.bsm_price = bsm_price

In [7]:
'''===============
Test bsm_price
================='''
gbm1 = Gbm(
    init_state = 100., 
    drift_ratio = .0475,
    vol_ratio = .2)
option1 = VanillaOption(
    otype = 1,
    strike = 110.,                
    maturity = 1.
)    

print('>>>>>>>>>>call value is ' + str(gbm1.bsm_price(option1)))
option2 = VanillaOption(otype=-1)
print('>>>>>>>>>>put value is ' + str(gbm1.bsm_price(option2)))

>>>>>>>>>>call value is 5.943273183452838
>>>>>>>>>>put value is 10.84042522804176


In the market, usually volatility (vol = 0.2 in the above) is not available, but market price (call = 5.94) for the option is available. One wants find a volatility such that associated BSM formula is equal to (fit) the market price. 

In [0]:
'''================
define an error function
===================='''
def error_function(vol, gbm, option):
  gbm.vol_ratio = vol
  return abs(option.market_price - gbm.bsm_price(option))

In [0]:
'''==========
define a method to seek for an implied volatility
============'''
import scipy.optimize as so
def implied_volatility(gbm, option):
  init_vol = .1 #initial guess
  return so.fmin(error_function, init_vol, 
                 args = (gbm, option), disp = 0)[0]

In [12]:
'''============
test the implied_vol by reversing bsm_formula example in the above
=============='''

option1.market_price = 5.94

print('>>>>>>>>implied volatility is ' + 
     str(implied_volatility(gbm1, option1)))


>>>>>>>>implied volatility is 0.19992187500000036
